In [1]:
import langchain
from langchain.agents import initialize_agent, load_tools, AgentType, Tool
from langchain.agents.react.base import DocstoreExplorer
from langchain.chains import RetrievalQA
from langchain.docstore import DocstoreFn
from langchain.document_loaders import ArxivLoader
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain.llms import LlamaCpp
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.prompts import PromptTemplate

In [2]:
# Globals
embedding_model = 'hkunlp/instructor-xl'
instruction_model_path = "/Users/bsantanna/dev/workspace/community/Llama-2-13b-chat-hf/ggml-model-f16.bin"
n_gpu_layers = 1
n_batch = 512
n_ctx = 4096

In [3]:
# initialize embeddings model
embedding = HuggingFaceInstructEmbeddings(model_name=embedding_model)

/Users/bsantanna/miniforge3/envs/ml/lib/python3.11/site-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


load INSTRUCTOR_Transformer
'NoneType' object has no attribute 'cadam32bit_grad_fp32'


/Users/bsantanna/miniforge3/envs/ml/lib/python3.11/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


max_seq_length  512


In [4]:
# Load LLM instruction following model
llm = LlamaCpp(
    model_path=instruction_model_path,
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    n_ctx=n_ctx,
    f16_kv=True,
    verbose=False,
)

llama.cpp: loading model from /Users/bsantanna/dev/workspace/community/Llama-2-13b-chat-hf/ggml-model-f16.bin
llama_model_load_internal: format     = ggjt v1 (pre #1405)
llama_model_load_internal: n_vocab    = 32000
llama_model_load_internal: n_ctx      = 4096
llama_model_load_internal: n_embd     = 5120
llama_model_load_internal: n_mult     = 6912
llama_model_load_internal: n_head     = 40
llama_model_load_internal: n_head_kv  = 40
llama_model_load_internal: n_layer    = 40
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: n_gqa      = 1
llama_model_load_internal: rnorm_eps  = 1.0e-06
llama_model_load_internal: n_ff       = 13824
llama_model_load_internal: freq_base  = 10000.0
llama_model_load_internal: freq_scale = 1
llama_model_load_internal: ftype      = 1 (mostly F16)
llama_model_load_internal: model size = 13B
llama_model_load_internal: ggml ctx size =    0.11 MB
llama_model_load_internal: mem required  = 25491.69 MB (+ 3200.00 MB per state)
llama_new_context

In [5]:
# Initialize documents
loader = ArxivLoader(query="dynamic scaling in self-similar systems", load_max_docs=20)
documents = loader.load()

In [6]:
# Initialize index
index = VectorstoreIndexCreator(
    embedding=embedding,
    vectorstore_cls=DocArrayInMemorySearch
).from_documents(documents)

In [7]:
# Initialize retriever QA
db = DocArrayInMemorySearch.from_documents(
    documents,
    embedding
)

# initialize retriever
retriever = db.as_retriever()

# initialize retrieval qa chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="map_reduce",
    retriever=retriever,
    verbose=True
)

In [32]:
# tools = load_tools(['llm-math'], llm=llm)
tools = [
    Tool(
        name="dynamic scaling in self-similar systems QA system",
        func=qa_chain.run,
        description="useful for when you need to ask information about dynamic scaling in self-similar systems",
    ),
]

In [33]:
# initialize agent
react = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [53]:
# Initialize prompt 
complete_prompt = PromptTemplate.from_template("""
You are a skilled scientist that likes solving difficult and challenging problems.

Follow these steps to effectively solve the multiple-choice problem enclosed within <>, considering answers that may vary in precision:

1. Learn from given context until you reach a comprehensive understanding of each concept in the question.
   
2. Break down the question to identify incremental steps that lead to the solution. Provide detailed reasoning throughout the process, including analysis of relevant concepts and cross-validation from multiple sources to ensure accuracy and confidence in your response.
   
3. Progressively work through the solution, focusing on increasing your confidence in the accuracy of options (A-E).

4. Assess and eliminate options (A-E) that are incorrect or inaccurate.

5. Validate the remaining options using a cause-and-effect approach along with necessary calculations, performing incremental steps.

6. Identify the correct option and return the corresponding letter (A-E). Before finalizing your answer, ensure your calculations substantiate its correctness.

7. Choose the option from (A-E) that most closely aligns with your conclusion based on the evidence and reasoning you have provided.

<
{question}

{options}
>""")

In [54]:
# Format input
question = """Which of the following is an accurate definition of dynamic scaling in self-similar systems?"""
options = """
A - Dynamic scaling refers to the evolution of self-similar systems, where data obtained from snapshots at fixed times exhibits similarity to the respective data taken from snapshots of any earlier or later time. This similarity is tested by a certain time-dependent stochastic variable x.

B - Dynamic scaling refers to the non-evolution of self-similar systems, where data obtained from snapshots at fixed times is similar to the respective data taken from snapshots of any earlier or later time. This similarity is tested by a certain time-dependent stochastic variable x.
	
C - Dynamic scaling refers to the evolution of self-similar systems, where data obtained from snapshots at fixed times is dissimilar to the respective data taken from snapshots of any earlier or later time. This dissimilarity is tested by a certain time-independent stochastic variable y.

D - Dynamic scaling refers to the non-evolution of self-similar systems, where data obtained from snapshots at fixed times is dissimilar to the respective data taken from snapshots of any earlier or later time. This dissimilarity is tested by a certain time-independent stochastic variable y.

E - Dynamic scaling refers to the evolution of self-similar systems, where data obtained from snapshots at fixed times is independent of the respective data taken from snapshots of any earlier or later time. This independence is tested by a certain time-dependent stochastic variable z.
"""
formatted_input = short_prompt.format_prompt(
    question=question,
    options=options
)

In [57]:
result = react(formatted_input)
print(result)



> Entering new AgentExecutor chain...
 I will use the dynamic scaling in self-similar systems QA system to gain comprehensive understanding of each concept and term presented in the question, make sure I have a deep familiarity with the problem at hand.
Action: I will break down the question to identify incremental steps that lead to the solution. Provide detailed reasoning throughout the process, including analysis of relevant concepts and cross-validation from multiple sources to ensure accuracy and confidence in my response.
Action Input: The question is broken down into the following incremental steps:
1. Define dynamic scaling in self-similar systems.
2. Identify the correct definition of dynamic scaling in self-similar systems.
3. Provide detailed reasoning for each option.
4. Assess options (A-E) and compare each option to my own solution.
5. Rank each option using a cause-and-effect approach along with necessary calculations.
6. Identify the correct option and return the corr